# Sports Downgrade Project

## Reading the data

We import some relevant packages:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
client = bigquery.Client()

We convert the data to a pandas dataframe (where the dates in the "DTV_Last_Activation_Dt" and "Sports_Last_Activation_Dt" columns have been changed to reflect the number of weeks between the current date and the activation date):

In [ ]:
table = """
    SELECT
        DATE_DIFF(CURRENT_DATE(), DTV_Last_Activation_Dt, week) AS DTV_Last_Activation_Dt,
        DATE_DIFF(CURRENT_DATE(), Sports_Last_Activation_Dt, week) AS Sports_Last_Activation_Dt,
        Offers_Applied_Ever_Sports,
        h_age_coarse,
        h_number_of_adults,
        h_number_of_children_in_hh,
        DTV_Product_Holding,
        Curr_Offer_Amount_Sports,
        Curr_Offer_Length_Sports,
        Target_sports_downgrade
    FROM
        `sky-uk-ids-analytics-prod.NPR13.Grad_Example_Propensity_Mart_Sports_Downgrades`
"""

data = client.query(table).to_dataframe()

Getting some information about the data:

In [ ]:
data.head()

In [ ]:
data.info()

We can see that there are many null values in the "Curr_Offer_Amount_Sports" and "Curr_Offer_Length_Sports" columns, which should be set to 0 by default. We address this:

In [ ]:
data["Curr_Offer_Amount_Sports"] = data["Curr_Offer_Amount_Sports"].fillna(0)
data["Curr_Offer_Length_Sports"] = data["Curr_Offer_Length_Sports"].fillna("0M")

We change the datatype of the "Curr_Offer_Length_Sports" column from 'string' to 'float':

In [ ]:
data["Curr_Offer_Length_Sports"] = data["Curr_Offer_Length_Sports"].map(lambda x: x.rstrip('M'))
data["Curr_Offer_Length_Sports"] = data["Curr_Offer_Length_Sports"].astype('float64')

## Visualising the data

### Numerical data

In [ ]:
data.hist(bins=30, figsize=(18, 15))

We can see that the "Curr_Offer_Length_Sports" column appears to contain some outlier values.

In [ ]:
print("Skew = {}".format(data["Curr_Offer_Length_Sports"].skew()))
data["Curr_Offer_Length_Sports"].describe()

We will use the IQR rule to detect the outliers and replace them with the median value:

In [ ]:
def outlier_detection(data, k=1.5):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_range = Q1 - (k * IQR)
    upper_range = Q3 + (k * IQR)
    return lower_range, upper_range

lower_range, upper_range = outlier_detection(data["Curr_Offer_Length_Sports"])
median = data["Curr_Offer_Length_Sports"].median()
mask = (data["Curr_Offer_Length_Sports"] < lower_range) | (data["Curr_Offer_Length_Sports"] > upper_range)
data.loc[mask, "Curr_Offer_Length_Sports"] = median

We now look at the new distribution of the "Curr_Offer_Length_Sports" column and confirm that there are no outlier values:

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
data["Curr_Offer_Length_Sports"].plot(kind="hist", bins=15)
plt.xlabel("Offer Length")
plt.title("Offer Length Distribution")
plt.grid()
plt.subplot(1, 2, 2)
data["Curr_Offer_Length_Sports"].plot(kind="box")
plt.ylabel("Offer Length")
plt.title("Offer Length Range")
plt.show()

We will now look at the correlation matrix of our dataset to see how different columns relate to each other:

In [ ]:
corr_matrix = data.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True)
plt.show()

We explore the relationship between the "Offers_Applied_Ever_Sports", "Current_Offer_Length_Sports" and "Current_Offer_Amount_Sports" columns, which appear to be correlated to some extent.

In [ ]:
from pandas.plotting import scatter_matrix

attribs = ["Offers_Applied_Ever_Sports", "Curr_Offer_Length_Sports", "Curr_Offer_Amount_Sports"]
scatter_matrix(data[attribs], figsize=(12, 8))

No apparent relationship can be derived from the scatterplots.

Lastly, we look at some descriptive statistics for the numerical columns:

In [ ]:
data.describe()

We remark that we are dealing with a heavily imbalanced dataset (the average of "Target_sports_downgrade" column is roughly 0.0016 i.e. only 0.16% of the customers in the dataset downgraded).

### Categorical data

We will split the data set into "downgraders" (rows that have Target_sports_downgrade = 1) and "non-downgraders" (rows that have Target_sports_downgrade = 0) and see how the two partitions compare across different categories.

In [ ]:
downgraders = data[data["Target_sports_downgrade"] == 1]
non_downgraders = data[data["Target_sports_downgrade"] == 0]

In [ ]:
data["DTV_Product_Holding"].value_counts().plot(kind="bar")
plt.title("Customer Distribution per DTV Product")
plt.ylabel("Number of customers")
plt.grid()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
non_downgraders["DTV_Product_Holding"].value_counts().plot(kind="pie", autopct='%1.1f%%')
plt.title("Non-downgraders Distribution per DTV Product")
plt.subplot(1, 2, 2)
downgraders["DTV_Product_Holding"].value_counts().plot(kind="pie", autopct='%1.1f%%')
plt.title("Downgraders Distribution per DTV Product")
plt.show()

In [ ]:
avg_downgrades_prod_q = """
    SELECT
        DTV_Product_Holding,
        AVG(Target_sports_downgrade) AS avg_downgrades
    FROM
        `sky-uk-ids-analytics-prod.NPR13.Grad_Example_Propensity_Mart_Sports_Downgrades`
    GROUP BY
        DTV_Product_Holding
    ORDER BY
        avg_downgrades DESC;
"""

avg_downgrades_prod = client.query(avg_downgrades_prod_q).to_dataframe()

In [ ]:
avg_downgrades_prod["avg_downgrades"].plot(kind="bar")
plt.axhline(y=0.001636, color='r', linestyle='--', label="Average downgrade rate")
pos = np.arange(8)
plt.xticks(pos, avg_downgrades_prod["DTV_Product_Holding"])
plt.title("Average Downgrade Rate per DTV Product")
plt.grid()

In [ ]:
data["h_age_coarse"].value_counts().plot(kind="bar")
plt.title("Customer Distribution per Age Level")
plt.ylabel("Number of customers")
plt.grid()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
(non_downgraders["h_age_coarse"].value_counts() / non_downgraders.shape[0]).plot(kind="bar")
plt.grid()
plt.title("Non-downgraders Distribution per Age Level")
plt.subplot(1, 2, 2)
(downgraders["h_age_coarse"].value_counts() / downgraders.shape[0]).plot(kind="bar")
plt.grid()
plt.title("Downgraders Distribution per Age Level")
plt.show()

In [ ]:
avg_downgrades_age_q = """
    SELECT
        AVG(Target_sports_downgrade) AS avg_downgrades,
        h_age_coarse
    FROM
        `sky-uk-ids-analytics-prod.NPR13.Grad_Example_Propensity_Mart_Sports_Downgrades`
    GROUP BY
        h_age_coarse
    ORDER BY
        avg_downgrades DESC
"""

avg_downgrades_age = client.query(avg_downgrades_age_q).to_dataframe()

In [ ]:
avg_downgrades_age["avg_downgrades"].plot(kind="bar")
plt.axhline(y=0.001636, color='r', linestyle='--')
pos = np.arange(7)
plt.xticks(pos, avg_downgrades_age["h_age_coarse"])
plt.title("Average Downgrade Rate per Age Level")
plt.grid()

## Pre-processing the data

The columns "Sports_Last_Activation_Dt" and "DTV_Last_Activation_Dt" are highly corellated, so we drop the latter.

In [ ]:
data = data.drop("DTV_Last_Activation_Dt", axis=1)

**OPTIONAL.** Binning the "Sports_Last_Activation_Dt" variable to improve modelling results:

In [ ]:
x = data["Sports_Last_Activation_Dt"].values
y = data["Target_sports_downgrade"].values

In [ ]:
from optbinning import OptimalBinning

optb = OptimalBinning(name="Sports_Last_Activation_Dt")
optb.fit(x, y)

In [ ]:
optb.binning_table.build()
optb.binning_table.plot(metric="woe", style="actual", add_special=False, add_missing=False)

In [ ]:
x_transform_woe = optb.transform(x, metric="woe")
df = pd.DataFrame(x_transform_woe, columns=["Sports_Last_Activation_Dt_Binned"])
data["Sports_Last_Activation_Dt"] = df["Sports_Last_Activation_Dt_Binned"]

We split the data into train and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(42)

train_set, test_set = train_test_split(data, test_size=0.2, stratify=data["Target_sports_downgrade"], random_state=42)

We drop the target variable from the train set:

In [ ]:
downgrades = train_set.drop("Target_sports_downgrade", axis=1)
downgrades_labels = train_set["Target_sports_downgrade"].copy()

Since we are dealing with a heavily imbalanced dataset, we use a combination of oversampling and undersampling techniques to improve modelling results:

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = RandomOverSampler(sampling_strategy=0.2)
under = RandomUnderSampler(sampling_strategy=0.8)
sampling_pipeline = Pipeline([("over_sampling", over), ("under_sampling", under)])

In [ ]:
downgrades, downgrades_labels = sampling_pipeline.fit_resample(downgrades, downgrades_labels)

We create a pipeline that prepares the data for modelling. We fill the missing values in the numerical fields with medians and scale the numerical variables and we encode categorical variables into numerical data:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_attribs = list(downgrades.select_dtypes(include=[np.number]))
cat_attribs = ["h_age_coarse", "DTV_Product_Holding"]

num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")),
                         ("std_scaler", StandardScaler())])

full_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs),
                                   ("cat", OneHotEncoder(), cat_attribs)])

In [ ]:
downgrades_prepared = full_pipeline.fit_transform(downgrades)

In [ ]:
downgrades_prepared.shape

## Modelling the data

We prepare the test data to measure the performance of our models:

In [ ]:
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.inspection import permutation_importance

X_test = test_set.drop("Target_sports_downgrade", axis=1)
X_test_prepared = full_pipeline.transform(X_test)
y_test = test_set["Target_sports_downgrade"].copy()

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(C=0.01, penalty="l1", solver="liblinear", max_iter=200)
log_reg.fit(downgrades_prepared, downgrades_labels)
pred_log_reg = log_reg.predict_proba(X_test_prepared)
print("AUC = {}".format(roc_auc_score(y_test, pred_log_reg[:, 1])))

In [ ]:
coef = log_reg.coef_[0]

results_log_reg = permutation_importance(log_reg, X_test_prepared, y_test, scoring="roc_auc")
importance_log_reg = results_log_reg.importances_mean

### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier(max_depth=6)
dec_tree.fit(downgrades_prepared, downgrades_labels)
pred_dec_tree = dec_tree.predict_proba(X_test_prepared)
print("AUC = {}".format(roc_auc_score(y_test, pred_dec_tree[:, 1])))

In [ ]:
feature_imp_dec_tree = dec_tree.feature_importances_

results_dec_tree = permutation_importance(dec_tree, X_test_prepared, y_test, scoring="roc_auc")
importance_dec_tree = results_dec_tree.importances_mean

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rand_for = RandomForestClassifier(n_jobs=-1, max_depth=6, n_estimators=200)
rand_for.fit(downgrades_prepared, downgrades_labels)
pred_rand_for = rand_for.predict_proba(X_test_prepared)
print("AUC = {}".format(roc_auc_score(y_test, pred_rand_for[:, 1])))

In [ ]:
feature_imp_rand_for = rand_for.feature_importances_

results_rand_for = permutation_importance(rand_for, X_test_prepared, y_test, scoring="roc_auc")
importance_rand_for = results_rand_for.importances_mean

### Comparing the models

ROC curves:

In [ ]:
classifiers = [log_reg, dec_tree, rand_for]

ax = plt.gca()
for classifier in classifiers:
    plot_roc_curve(classifier, X_test_prepared, y_test, ax=ax)

Lift on first decile:

In [ ]:
def sorted_ind(y_proba):
    y_proba_sorted_ind = sorted(range(len(y_proba)), key=lambda k: y_proba[k])
    return y_proba_sorted_ind[::-1]

def first_decile_recall(y_test, y_proba_sorted_ind):
    s = 0
    for i in range(int(len(y_test) / 10)):
        s += y_test.iloc[y_proba_sorted_ind[i]]
    return s / int(len(y_test) / 10)

In [ ]:
log_reg_proba = pred_log_reg[:, 1]
log_reg_proba_sorted_ind = sorted_ind(log_reg_proba)
dec_tree_proba = pred_dec_tree[:, 1]
dec_tree_proba_sorted_ind = sorted_ind(dec_tree_proba)
rand_for_proba = pred_rand_for[:, 1]
rand_for_proba_sorted_ind = sorted_ind(rand_for_proba)
base_rate = sum(y_test) / len(y_test)
print("Base rate for test dataset: {}%".format(base_rate))
print("First decile lift Logistic Regression:" 
      + " {}".format(first_decile_recall(y_test, log_reg_proba_sorted_ind) / base_rate))
print("First decile lift Decision Tree:"
     + " {}.".format(first_decile_recall(y_test, dec_tree_proba_sorted_ind) / base_rate))
print("First decile lift Random Forest:"
     + " {}".format(first_decile_recall(y_test, rand_for_proba_sorted_ind) / base_rate))

**NOTE.** Better results can be obtained if the oversampling/undersampling pipeline is not used for the train data.

Feature importance:

In [ ]:
labels = np.concatenate([num_attribs, full_pipeline.named_transformers_["cat"].get_feature_names(cat_attribs)])
for ind, label in enumerate(labels):
    print("Feature #{}: {}".format(ind, label))

In [ ]:
plt.figure(figsize=(13, 3))
plt.subplot(1, 3, 1)
plt.bar([_ for _ in range(len(coef))], coef)
plt.grid()
plt.title("Logistic Regression Feature Coefficients")
plt.subplot(1, 3, 2)
plt.bar([_ for _ in range(len(feature_imp_dec_tree))], feature_imp_dec_tree)
plt.grid()
plt.title("Decision Tree Feature Importance")
plt.subplot(1, 3, 3)
plt.bar([_ for _ in range(len(feature_imp_rand_for))], feature_imp_rand_for)
plt.grid()
plt.title("Random Forest Feature Importance")
plt.show()

In [ ]:
plt.figure(figsize=(13, 3))
plt.subplot(1, 3, 1)
plt.bar([_ for _ in range(len(importance_log_reg))], importance_log_reg)
plt.grid()
plt.title("Logistic Regression Permutation Importance")
plt.subplot(1, 3, 2)
plt.bar([_ for _ in range(len(importance_dec_tree))], importance_dec_tree)
plt.grid()
plt.title("Decision Tree Permutation Importance")
plt.subplot(1, 3, 3)
plt.bar([_ for _ in range(len(importance_rand_for))], importance_rand_for)
plt.grid()
plt.title("Random Forest Permutation Importance")
plt.show()

Inspecting all these charts, we can see that across all of the 3 models, the most important features are "Sports_Last_Activation_Dt", "Curr_Offer_Length_Sports" and "DTV_Product_Holding_Sky Entertainment".

## --- DRAFT ---

Grid search for hyperparameters (with CV):

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV

model = LogisticRegression()
solvers = ["liblinear"]
penalty = ["l2", "l1"]
c_values = [10, 1, 0.1, 0.01]

In [ ]:
grid = dict(solver=solvers, penalty=penalty, C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring="roc_auc", error_score=0)

In [ ]:
grid_result = grid_search.fit(downgrades_prepared, downgrades_labels)

In [ ]:
print("Best: {} using {}".format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
model = RandomForestClassifier()
n_estimators = [50, 100, 200]
max_depth = [4, 5, 6, 10]

In [ ]:
grid = dict(n_estimators=n_estimators, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring="roc_auc", error_score=0)

In [ ]:
grid_result = grid_search.fit(downgrades_prepared, downgrades_labels)

In [ ]:
print("Best: {} using {}".format(grid_result.best_score_, grid_result.best_params_))

Recursive feature elimination (with CV):

In [ ]:
from sklearn.feature_selection import RFECV

selector = RFECV(log_reg, step=1)
selector = selector.fit(downgrades_prepared, downgrades_labels)

In [ ]:
selector.ranking_

In [ ]:
new_X_train = downgrades_prepared[:, selector.support_]
new_X_test = X_test_prepared[:, selector.support_]

In [ ]:
log_reg_upd = LogisticRegression(C=0.01, penalty="l1", solver="liblinear", max_iter=200)
log_reg_upd.fit(new_X_train, downgrades_labels)
pred_log_reg_upd = log_reg_upd.predict_proba(new_X_test)
print("AUC = {}".format(roc_auc_score(y_test, pred_log_reg_upd[:, 1])))